# First Aid Recommendation ChatBot

this bot helps you find treatment for your case quickly

In [94]:
# importing libraries
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

let me try to describe how to do **chatbot** as an idea 

first, we need to find data and separate it into `inputs` and `outputs`
user will give me a **sentence** 
fine the `inputs` of the model may be lots of `sentences` 
and of course, I need labels to act each sentence to use it as the target 
the response of the bot may be any answer **connected** with the labels and the sentences 
let's try it step by step 

data collected from Kaggle competition (**First Aid Recommendations Intents**)

this data is structured so I used it 

it has

tag >> labels

patterns >> sentence input 

responses >> sentence output of chat


In [95]:

intents={"intents": [    
        {"tag": "greeting",
         "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
         "responses": ["Hello", "Hi", "Hi there"]
        },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye"],
         "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
        },
        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
         "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
        },
        {"tag": "about",
         "patterns": ["Who are you?", "What are you?", "Who you are?" ],
         "responses": ["I.m ammar_bot, your ammar_bot assistant", "I'm ammar_bot, an Artificial Intelligent bot"]
        },
        {"tag": "name",
        "patterns": ["what is your name", "what should I call you", "whats your name?"],
        "responses": ["You can call me ammar_bot.", "I'm ammar_bot!", "Just call me as ammar_bot"]
        },
        {"tag": "help",
        "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
        "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
        },
        {"tag": "createaccount",
        "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
        "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
        },
        {"tag": "complaint",
        "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
        "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
        },

        
        {"tag": "Cuts",
         "patterns": ["What to do if Cuts?", "How to cure Cuts?", "Which medicine to apply for Cuts?", "what to apply on cuts?", "Cuts"],
         "responses": ["Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied."],
         "context_set": ""
        },

        {"tag": "Abrasions",
         "patterns": ["how do you treat abrasions?", "Do Abrasions cause scars?", "Abrasions", "what to do if abrasions?", "Which medicine to apply for abrasions?", "How to cure abrasions?"],
         "responses": ["Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap. Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape that’s not bleeding, leave the wound uncovered.If the wound is bleeding, use a clean cloth or bandage, and apply gentle pressure to the area to stop any bleeding.Cover a wound that bled with a thin layer of topical antibiotic ointment, like Bacitracin, or a sterile moisture barrier ointment, like Aquaphor. Cover it with a clean bandage or gauze. Gently clean the wound and change the ointment and bandage once per day.Watch the area for signs of infection, like pain or redness and swelling. See your doctor if you suspect infection."],
         "context_set": ""
        },

        {"tag": "stings",
         "patterns": ["How do you treat Sting?", "Stings", "What to do if you get a sting?", "Which medicine to apply if sting?"],
         "responses": ["Remove any stingers immediately. Some experts recommend scraping out the stinger with a credit card. Applying ice to the site may provide some mild relief. Apply ice for 20 minutes once every hour as needed. Wrap the ice in a towel or keep a cloth between the ice and skin to keep from freezing the skin. Taking an antihistamine such as diphenhydramine (Benadryl) or a nonsedating one such as loratadine (Claritin) will help with itching and swelling. Take acetaminophen (Tylenol) or ibuprofen (Motrin)for pain relief as needed. Wash the sting site with soap and water. Placing hydrocortisone cream on the sting can help relieve redness, itching, and swelling."],
         "context_set": ""
        },

        {"tag": "Splinter",
         "patterns": ["How to remove Splinters", "How to cure Splinters?", "What to do if I have splinters?", "How do you bring a splinter to the surface?"],
         "responses": ["1. SOAK IT IN EPSOM SALTS. Dissolve a cup of the salts into a warm bath and soak whatever part of the body has the splinter. Failing that, you can also put some of the salts onto a bandage pad and leave it covered for a day; this will eventually help bring the splinter to the surface. 2. VINEGAR OR OIL. Another simple way to draw out that stubborn splinter is to soak the affected area in oil (olive or corn) or white vinegar. Just pour some in a bowl and soak the area for around 20 to 30 minutes,"],
         "context_set": ""
        },

        {"tag": "Sprains",
         "patterns": ["How do you treat a sprain?", "what to do if i get a sprain?", "Which cream to apply if i get a sprain?", "Which medicine to apply if I get a sprain?"],
         "responses": ["Use an ice pack or ice slush bath immediately for 15 to 20 minutes and repeat every two to three hours while you're awake. To help stop swelling, compress the ankle with an elastic bandage until the swelling stops. In most cases, pain relievers — such as ibuprofen (Advil, Motrin IB, others) or naproxen sodium (Aleve, others) or acetaminophen (Tylenol, others) are enough to manage the pain of a sprained ankle."],
         "context_set": ""
        },

        {"tag": "Strains",
         "patterns": ["How do you treat a strain?", "what to do if i get a strain?", "Which cream to apply if i get a strain?", "Which medicine to apply if I get a strain?", "How do you diagnose a strain?", "Is heat or ice better for a pulled muscle?"],
         "responses": ["Rest,Ice,Compression and Elevation can be used to cure strains. Avoid using your muscle for a few days, especially if movement causes an increase in pain and also Apply ice immediately after injuring your muscle. This will minimize swelling. Don’t put ice directly on your skin. Use an ice pack or wrap ice in a towel. To reduce swelling, wrap the affected area with an elastic bandage until swelling comes down."],
         "context_set": ""
        },

        {"tag": "Fever",
         "patterns": ["How do you treat a mild Fever?", "what to do if i get a mild fever?", "Which medicine to take if I get a mild fever?", "fever"],
         "responses": ["To treat a fever at home: 1)Drink plenty of fluids to stay hydrated. 2)Dress in lightweight clothing. 3)Use a light blanket if you feel chilled, until the chills end. 4)Take acetaminophen (Tylenol, others) or ibuprofen (Advil, Motrin IB, others). 5) Get medical help if the fever lasts more than five days in a row."],
         "context_set": ""
        },

        {"tag": "Nasal Congestion",
         "patterns": ["How do you treat nasal Congestion?", "what to do if i get a nasal congestion?", "Which medicine to take if I have a nasal congestion?", "what to do if i have a blocked nose?", "How do you treat a blocked nose?", "How long does nasal congestion last?"],
         "responses": ["When you’re stuffed up, focus on keeping your nasal passages and sinuses moist. To keep your nasal passages moist, you can: 1)Use a humidifier or vaporizer. 2)Drink lots of fluids. This will thin out your mucus, which could help prevent blocked sinuses. 3)Place a warm, wet towel on your face. It may relieve discomfort and open your nasal passages."],
         "context_set": ""
        },

        {"tag": "Cough",
         "patterns": ["How to cure cough?","How do you treat cough?", "what to do if i get a cough?", "Which medicine to take if I get cough?", "How do you get rid of cough?"],
         "responses": ["1) Honey:- Use honey to treat a cough, mix 2 teaspoons (tsp) with warm water or an herbal tea. Drink this mixture once or twice a day. Do not give honey to children under 1 year of age. 2) Ginger:- Brew up a soothing ginger tea by adding 20–40 grams (g) of fresh ginger slices to a cup of hot water. Allow to steep for a few minutes before drinking. Add honey or lemon juice to improve the taste and further soothe a cough. 3) Fluids:- Staying hydrated is vital for those with a cough or cold. Research indicates that drinking liquids at room temperature can alleviate a cough, runny nose, and sneezing."],
         "context_set": ""
        },

        {"tag": "Sore Throat",
         "patterns": ["How do you treat sore throat?", "what to do if i get a sore throat?", "Which medicine to take if I get a sore throat?", "How to cure sore throat?"],
         "responses": ["1) Make sure you get plenty of rest and drink a lot of fluids. 2)Inhale steam,Run hot water in a sink.Drape a towel over you to trap the steam, and have the person lean over the sink with the water running. Tell him to breathe deeply through his mouth and nose for 5 to 10 minutes. Repeat several times a day. 3)Have him sip chicken broth or warm tea with honey. Don’t give honey to a child under 12 months of age."],
         "context_set": ""
        },

        {"tag": "Gastrointestinal problems",
         "patterns": ["How do you treat gas problems?", "what to do if i have Gastrointestinal problems?", "Which medicine to take if I get gas problem?", "How to cure Gas problems?"],
         "responses": ["1) Replenish body fluids 2)Do not take antidiarrheal drugs or laxatives or pain medication, unless specified by a medical professional.3)Taking antacids may help, per recommendation of a healthcare professional. 4)If prone to frequent heartburns, seek medical help. 5)Taking meals that are not spicy regularly, can relieve ulcer pains. 6)Seek medical help, if conditions persist or continue to worsen"],
         "context_set": ""
        },

        {"tag": "Skin problems",
         "patterns": ["How do you treat Skin problems?", "what to do if i get a skin allergy?", "Which medicine to take if I get a skin allergy?", "How to cure skin allergy?"],
         "responses": ["1)Hydrocortisone cream. 2)Ointments like calamine lotion. 3)Antihistamines. 4)Cold compresses. 5)Oatmeal baths. 6)Talk to your doctor about what's best for your specific rash."],
         "context_set": ""
        },

        {"tag": "Abdonominal Pain",
         "patterns": ["How do you treat Abdonominal Pain?", "what to do if i get a Abdonominal Pain?", "Which medicine to take if I get a Abdonominal Pain?", "How to cure Abdonominal Pain?"],
         "responses": ["1)Provide clear fluids to sip, such as water, broth, or fruit juice diluted with water. 2)Serve bland foods, such as saltine crackers, plain bread, dry toast, rice, gelatin, or applesauce. 3)Avoid spicy or greasy foods and caffeinated or carbonated drinks until 48 hours after all symptoms have gone away. "],
         "context_set": ""
        },

        {"tag": "Bruises",
         "patterns": ["How do you treat Bruises?", "what to do if i get a Bruise?", "Which medicine to take if I get a Bruise?", "How to cure Bruises?"],
         "responses": ["1)Ice the bruise with an ice pack wrapped in a towel. 2)Leave it in place for 10 to 20 minutes. 3)Repeat several times a day for a day or two as needed. 4)Compress the bruised area if it is swelling, using an elastic bandage. "],
         "context_set": ""
        },

        {"tag": "Broken Toe",
         "patterns": ["How do you treat a Broken Toe?", "what to do if i get a Broken Toe?", "Which medicine to take if I get a Broken Toe?", "How to cure Broken Toe?"],
         "responses": ["1)To help decrease pain and swelling in a broken toe, elevate the foot, ice the injury, and stay off the foot. 2)Depending on the severity of the fracture, the toe may need to be put back into place (reduced), and some compound toe fractures may require surgery.3) Most broken toes heal without complications in six weeks."],
         "context_set": ""
        },

        {"tag": "Choking",
         "patterns": ["How do you treat Choking?", "what to do if i get a Choke?", "Which medicine to take if I get Choked?", "How to cure Choking?"],
         "responses": ["1)Encourage them to keep coughing to try to clear the blockage. 2)Ask them to try to spit out the object if it's in their mouth. 3)If coughing doesn't work, start back blows and Abdonominal thrusts"],
         "context_set": ""
        },

        {"tag": "Wound",
         "patterns": ["How do you treat a wound?", "what to do if i get a Wound?", "Which medicine to take if I get wounded?", "How to cure a wound?"],
         "responses": ["1)Rinse the cut or wound with water and apply pressure with sterile gauze, a bandage, or a clean cloth. 2)If blood soaks through the bandage, place another bandage on top of the first and keep applying pressure. 3)Raise the injured body part to slow bleeding. 4)When bleeding stops, cover the wound with a new, clean bandage."],
         "context_set": ""
        },

        {"tag": "Diarrhea",
         "patterns": ["How do you treat Diarrhea?", "what to do if i get Diarrhea?", "Which medicine to take if I get Diarrhea?", "How to cure Diarrhea?"],
         "responses": ["1)Hydrating the body is essential for recovering from diarrhea.This causes the body to lose electrolytes such as sodium and chloride. 2)It is highly recommended to avoid dairy products, as they may worsen diarrhea in some people. 3)However, if diarrhea lasts for more than 2 days, seek medical advice to avoid complications."],
         "context_set": ""
        },

        {"tag": "Frost bite",
         "patterns": ["How do you treat a Frost bite?", "what to do if i get a Frost bite?", "Which medicine to take if I get a Frost bite?", "How to cure Frost bite?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Heat Exhaustion",
         "patterns": ["How do you treat Heat Exhaustion?", "what to do if i feel Exhausted due to heat?", "Which medicine to take if I get Exhausted?", "How to cure Heat Exhaustion?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Heat Stroke",
         "patterns": ["How do you treat Heat Stroke?", "what to do if i get a Heat Stroke?", "Which medicine to take if I get Stroke?", "How to cure a Heat Stroke?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Insect Bites",
         "patterns": ["How do you treat a Insect Bite?", "what to do if a insect bites me?", "Which medicine to take if I get bitten by a insect?", "How to cure insect bite?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "nose bleed",
         "patterns": ["How do you treat a bleeding nose?", "what to do if i my nose is bleeding?", "Which medicine to take if I get nose bleed?", "How to cure nose bleeding?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Pulled Muscle",
         "patterns": ["How do you treat a Pulled Muscle?", "what to do if my muscle is pulled?", "Which medicine to take if I got pulled muscle?", "How to cure a pulled muscle?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Rectal bleeding",
         "patterns": ["How do you treat Rectal Bleeding?", "what to do if i get a Rectal Bleeding?", "Which medicine to take if I get Rectal Bleeding?", "How to cure Rectal Bleeding?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Sun Burn",
         "patterns": ["How do you treat Sun Burn?", "what to do if i get a Sun Burn?", "Which medicine to take if I get Sun Burn?", "How to cure a Sun Burn?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Testicle Pain",
         "patterns": ["How do you treat Testicle Pain?", "what to do if i get a Testicle Pain?", "Which medicine to take if I get a Testicle Pain?", "How to cure Testicle Pain?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Vertigo",
         "patterns": ["How do you treat a Vertigo?", "what to do if i get a Vertigo?", "Which medicine to take if I get Vertigo?", "How to cure a Vertigo?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Normal Bleeding",
         "patterns": ["How do you treat bleeding?", "what to do if i get a Bleeding?", "Which medicine to take if I get bleeding?", "How to cure Bleeding?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Eye Injury",
         "patterns": ["How do you treat an eye Injury?", "what to do if i get a eye Injury?", "Which medicine to take if I injured my eye?", "How to cure injured eye?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Chemical Burn",
         "patterns": ["How do you treat a chemical burn?", "what to do if i get a Chemical Burn?", "Which medicine to take if I get burn due to chemicals?", "How to cure Chemical Burn?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Poison",
         "patterns": ["How do you treat a Poison?", "what to do if i get Poison?", "Which medicine to take if I am poisoned?", "How to cure Poisoning?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Teeth",
         "patterns": ["How do you treat broken Teeth ?", "what to do if my Teeth got broken?", "Which medicine to take if I get broken teeth?", "cure broken teeth?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "seizure",
         "patterns": ["How do you treat a seizure?", "what to do if i get a seizure?", "Which medicine to take if I get seizure?", "How to cure seizure?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Head Injury",
         "patterns": ["How do you treat a head Injury?", "what to do if i get a Head Injury?", "Which medicine to take if I get injured in the head?", "How to cure Head Injury?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Fainting",
         "patterns": ["How do you treat Faint?", "what to do if i feel like Fainting?", "Which medicine to take if I get a Faint?", "How to cure Fainting?"],
         "responses": [" "],
         "context_set": ""
        },

        {"tag": "Headache",
         "patterns": ["How do you treat a mild Headache?", "what to do if i get a mild Headache?", "Which medicine to take if I have a mild headache?", "How to cure a mild headache?"],
         "responses": ["Give ibuprofen (Advil, Motrin), aspirin, or acetaminophen (Tylenol) for pain. Avoid ibuprofen and other NSAIDs if the person has heart failure or kidney failure. Do not give aspirin to a child under age 18."],
         "context_set": ""
        },

        {"tag": "Cold",
         "patterns": ["How do you treat a Cold?", "what to do if i get a mild Cold?", "Which medicine to take if I have a Cold?", "How to cure Cold?"],
         "responses": ["1)Keeping hydrated is absolutely vital to help 'flush' out the cold, as well as to break down congestion and keep your throat lubricated. 2)Vitamin C is extremely helpful when fighting infection, so at the first sign of a cold be sure to increase your intake by eating plenty of berries, citrus fruits, papayas, broccoli and red peppers which will help keep you protected. 3)When it comes to combating a cold,Vitamin D is essential in helping to regulate immune response."],
         "context_set": ""
        },

        {"tag": "Rash",
         "patterns": ["How do you treat Rashes?", "what to do if i get a Rash?", "Which medicine to take if I have a Rash?", "How to cure Rash?"],
         "responses": ["1)Olive oil helps in healing and promotes skin renewal given it is packed with vitamin E and antioxidants. It also soothes the skin and reduces itching. 2)Baking soda is useful in drying skin rashes as also in relieving itching and inflammation. 3)Aloe Vera,Thanks to its antibacterial, antifungal, anti-inflammatory and emollient properties, aloe vera is excellent for treating a number of skin ailments including rashes as also soothing the skin."],
         "context_set": ""
        },

        {"tag": "snake bite",
         "patterns": ["How do you treat a snake bite?", "what to do if i get a snake bite?", "Which medicine to take if I get a snake bite?", "How to cure snake bite?", "i got bit by a snake"],
         "responses": ["While waiting for medical help: 1)Move the person beyond striking distance of the snake. 2)Have the person lie down with wound below the heart. 3)Keep the person calm and at rest, remaining as still as possible to keep venom from spreading. 4)Cover the wound with loose, sterile bandage. 5)Remove any jewelry from the area that was bitten. 6)Remove shoes if the leg or foot was bitten."],
         "context_set": ""
        },

        {"tag": "animal bite",
         "patterns": ["How do you treat a animal bite?", "How do you treat a monkey bite?","How do you treat a dog bite?", "what to do if i get a animal bite?", "Which medicine to take if I get a monekey bite?", "How to cure dog bite?", "i got bit by a dog"],
         "responses": ["1)Wash the wound with soap and warm water. 2)Gently press a clean cloth over the wound to stop the flow of blood. 3)Apply an antibacterial ointment to the wound. 4)Cover with a sterile bandage. 5)Watch for signs of infection. 6)Seek help if you suspect infection or possible exposure to rabies, or if the wound is severe."],
         "context_set": ""
        },

        {"tag": "Drowning",
         "patterns": ["What to do if someone is Drowning?", "what to do if someone drowned?", "What steps to take if i see a drowning person?", "How to help a drowning person?"],
         "responses": ["1)Place your ear next to the person's mouth and nose. Do you feel air on your cheek? 2)Look to see if the person's chest is moving.If the Person is Not Breathing, Check Pulse. 3)Check the person's pulse for 10 seconds.If There is No Pulse, Start CPR."],
         "context_set": ""
        },

    {"tag": "CPR",
         "patterns": ["How to give CPR??", "what to do in a CPR?", "What steps to take in a CPR??", "How to help a drowning person in CPR?"],
         "responses": ["1)For an adult or child, place the heel of one hand on the center of the chest at the nipple line. You can also push with one hand on top of the other. For an infant, place two fingers on the breastbone. 2)For an adult or child, press down at least 2 inches. Make sure not to press on ribs. For an infant, press down about 1 and 1/2 inches. Make sure not to press on the end of the breastbone. 3)Do chest compressions only, at the rate of 100-120 per minute or more. Let the chest rise completely between pushes. 4)Check to see if the person has started breathing."],
         "context_set": ""
        },

        {"tag": "Fracture",
         "patterns": ["How do you treat a Fracture?", "what to do if i get a Fracture?", "Which medicine to take if I have a Fracture?", "How to cure a Fracture?"],
         "responses": ["1)Stop any bleeding. Apply pressure to the wound with a sterile bandage, a clean cloth or a clean piece of clothing. 2)Immobilize the injured area. Don't try to realign the bone or push a bone that's sticking out back in. If you've been trained in how to splint and professional help isn't readily available, apply a splint to the area above and below the fracture sites. Padding the splints can help reduce discomfort. 3)Apply ice packs to limit swelling and help relieve pain. Don't apply ice directly to the skin. Wrap the ice in a towel, piece of cloth or some other material. 4)Treat for shock. If the person feels faint or is breathing in short, rapid breaths, lay the person down with the head slightly lower than the trunk and, if possible, elevate the legs."],
         "context_set": ""
        }
   ]
}
with open("intents.json", "w") as file:
    json.dump(intents, file)

# load the json file and extract the data.

In [96]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        # get ach sentence
        training_sentences.append(pattern)
        #get each tag of each sentence 
        training_labels.append(intent['tag'])
    # get all responses of a sentence
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        # get unique tags
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [112]:
training_sentences[:20]

['Hi',
 'Hey',
 'Is anyone there?',
 'Hello',
 'Hay',
 'Bye',
 'See you later',
 'Goodbye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Thanks for the help',
 'Who are you?',
 'What are you?',
 'Who you are?',
 'what is your name',
 'what should I call you',
 'whats your name?',
 'Could you help me?',
 'give me a hand please']

In [105]:
training_labels

array([44, 44, 44, 44, 44, 43, 43, 43, 51, 51, 51, 51, 39, 39, 39, 46, 46,
       46, 45, 45, 45, 45, 45, 45, 45, 42, 42, 42, 42, 42, 41, 41, 41,  9,
        9,  9,  9,  9,  1,  1,  1,  1,  1,  1, 50, 50, 50, 50, 31, 31, 31,
       31, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33, 14, 14, 14, 14, 23, 23,
       23, 23, 23, 23,  8,  8,  8,  8,  8, 30, 30, 30, 30, 17, 17, 17, 17,
       29, 29, 29, 29,  0,  0,  0,  0,  3,  3,  3,  3,  2,  2,  2,  2,  6,
        6,  6,  6, 38, 38, 38, 38, 10, 10, 10, 10, 16, 16, 16, 16, 20, 20,
       20, 20, 21, 21, 21, 21, 22, 22, 22, 22, 47, 47, 47, 47, 26, 26, 26,
       26, 28, 28, 28, 28, 34, 34, 34, 34, 36, 36, 36, 36, 37, 37, 37, 37,
       24, 24, 24, 24, 12, 12, 12, 12,  5,  5,  5,  5, 25, 25, 25, 25, 35,
       35, 35, 35, 48, 48, 48, 48, 18, 18, 18, 18, 13, 13, 13, 13, 19, 19,
       19, 19,  7,  7,  7,  7, 27, 27, 27, 27, 49, 49, 49, 49, 49, 40, 40,
       40, 40, 40, 40, 40, 11, 11, 11, 11,  4,  4,  4,  4, 15, 15, 15, 15])

In [114]:
responses[:10]

[['Hello', 'Hi', 'Hi there'],
 ['See you later', 'Have a nice day', 'Bye! Come back again'],
 ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"],
 ['I.m ammar_bot, your ammar_bot assistant',
  "I'm ammar_bot, an Artificial Intelligent bot"],
 ['You can call me ammar_bot.', "I'm ammar_bot!", 'Just call me as ammar_bot'],
 ['Tell me how can assist you',
  'Tell me your problem to assist you',
  'Yes Sure, How can I support you'],
 ['You can just easily create a new account from our web site',
  'Just go to our web site and follow the guidelines to create a new account'],
 ['Please provide us your complaint in order to assist you',
  'Please mention your complaint, we will reach you and sorry for any inconvenience caused'],
 ['Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage.

Then we use `LabelEncoder()` function provided by scikit-learn to convert the target labels into a model understandable form.

In [97]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [115]:
training_labels[:20]

array([44, 44, 44, 44, 44, 43, 43, 43, 51, 51, 51, 51, 39, 39, 39, 46, 46,
       46, 45, 45])

 we vectorize our text data corpus by using the `Tokenizer` class

In [98]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [117]:
padded_sequences[:10]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0, 115],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0, 116],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,  24, 117,  88],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0, 118],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0, 119],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0, 120],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,  89,   9, 121],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0, 122],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  

`pad_sequences` transform all sentences to be equal in vetors

# Model Training
Let’s define our Neural Network architecture for the proposed model and for that we use the “Sequential” model class of Keras

In [99]:

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_5   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_15 (Dense)            (None, 16)                272       
                                                                 
 dense_16 (Dense)            (None, 16)                272       
                                                                 
 dense_17 (Dense)            (None, 52)                884       
                                                                 
Total params: 17,428
Trainable params: 17,428
Non-trainable params: 0
__________________________________________________

fit the model with `padded_sequences` and `training_labels`

In [100]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
7/7 [==============================] - 1s 5ms/step - loss: 3.9522 - accuracy: 0.0136
Epoch 2/500
7/7 [==============================] - 0s 6ms/step - loss: 3.9510 - accuracy: 0.0271
Epoch 3/500
7/7 [==============================] - 0s 6ms/step - loss: 3.9502 - accuracy: 0.0181
Epoch 4/500
7/7 [==============================] - 0s 6ms/step - loss: 3.9495 - accuracy: 0.0226
Epoch 5/500
7/7 [==============================] - 0s 6ms/step - loss: 3.9487 - accuracy: 0.0271
Epoch 6/500
7/7 [==============================] - 0s 5ms/step - loss: 3.9480 - accuracy: 0.0407
Epoch 7/500
7/7 [==============================] - 0s 6ms/step - loss: 3.9470 - accuracy: 0.0452
Epoch 8/500
7/7 [==============================] - 0s 6ms/step - loss: 3.9463 - accuracy: 0.0498
Epoch 9/500
7/7 [==============================] - 0s 5ms/step - loss: 3.9451 - accuracy: 0.0498
Epoch 10/500
7/7 [==============================] - 0s 5ms/step - loss: 3.9439 - accuracy: 0.0362
Epoch 11/500
7/7 [===========

7/7 [==============================] - 0s 6ms/step - loss: 2.3422 - accuracy: 0.3484
Epoch 85/500
7/7 [==============================] - 0s 6ms/step - loss: 2.3086 - accuracy: 0.3575
Epoch 86/500
7/7 [==============================] - 0s 5ms/step - loss: 2.2751 - accuracy: 0.3756
Epoch 87/500
7/7 [==============================] - 0s 6ms/step - loss: 2.2458 - accuracy: 0.3801
Epoch 88/500
7/7 [==============================] - 0s 5ms/step - loss: 2.2129 - accuracy: 0.3937
Epoch 89/500
7/7 [==============================] - 0s 5ms/step - loss: 2.1826 - accuracy: 0.4118
Epoch 90/500
7/7 [==============================] - 0s 5ms/step - loss: 2.1497 - accuracy: 0.4118
Epoch 91/500
7/7 [==============================] - 0s 6ms/step - loss: 2.1172 - accuracy: 0.4525
Epoch 92/500
7/7 [==============================] - 0s 5ms/step - loss: 2.0895 - accuracy: 0.4842
Epoch 93/500
7/7 [==============================] - 0s 6ms/step - loss: 2.0583 - accuracy: 0.5023
Epoch 94/500
7/7 [===============

7/7 [==============================] - 0s 4ms/step - loss: 0.6628 - accuracy: 0.9502
Epoch 168/500
7/7 [==============================] - 0s 4ms/step - loss: 0.6516 - accuracy: 0.9548
Epoch 169/500
7/7 [==============================] - 0s 6ms/step - loss: 0.6434 - accuracy: 0.9548
Epoch 170/500
7/7 [==============================] - 0s 5ms/step - loss: 0.6347 - accuracy: 0.9412
Epoch 171/500
7/7 [==============================] - 0s 5ms/step - loss: 0.6244 - accuracy: 0.9502
Epoch 172/500
7/7 [==============================] - 0s 4ms/step - loss: 0.6160 - accuracy: 0.9548
Epoch 173/500
7/7 [==============================] - 0s 4ms/step - loss: 0.6078 - accuracy: 0.9367
Epoch 174/500
7/7 [==============================] - 0s 4ms/step - loss: 0.5971 - accuracy: 0.9729
Epoch 175/500
7/7 [==============================] - 0s 4ms/step - loss: 0.5926 - accuracy: 0.9774
Epoch 176/500
7/7 [==============================] - 0s 4ms/step - loss: 0.5823 - accuracy: 0.9683
Epoch 177/500
7/7 [=====

7/7 [==============================] - 0s 5ms/step - loss: 0.2062 - accuracy: 0.9955
Epoch 250/500
7/7 [==============================] - 0s 5ms/step - loss: 0.2036 - accuracy: 0.9955
Epoch 251/500
7/7 [==============================] - 0s 4ms/step - loss: 0.2008 - accuracy: 0.9955
Epoch 252/500
7/7 [==============================] - 0s 4ms/step - loss: 0.1980 - accuracy: 0.9955
Epoch 253/500
7/7 [==============================] - 0s 4ms/step - loss: 0.1947 - accuracy: 0.9955
Epoch 254/500
7/7 [==============================] - 0s 4ms/step - loss: 0.1918 - accuracy: 0.9955
Epoch 255/500
7/7 [==============================] - 0s 5ms/step - loss: 0.1887 - accuracy: 0.9955
Epoch 256/500
7/7 [==============================] - 0s 5ms/step - loss: 0.1863 - accuracy: 0.9955
Epoch 257/500
7/7 [==============================] - 0s 5ms/step - loss: 0.1835 - accuracy: 0.9955
Epoch 258/500
7/7 [==============================] - 0s 5ms/step - loss: 0.1822 - accuracy: 0.9955
Epoch 259/500
7/7 [=====

7/7 [==============================] - 0s 5ms/step - loss: 0.0666 - accuracy: 1.0000
Epoch 332/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0659 - accuracy: 1.0000
Epoch 333/500
7/7 [==============================] - 0s 6ms/step - loss: 0.0653 - accuracy: 1.0000
Epoch 334/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0641 - accuracy: 1.0000
Epoch 335/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0636 - accuracy: 1.0000
Epoch 336/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0625 - accuracy: 1.0000
Epoch 337/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0617 - accuracy: 1.0000
Epoch 338/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0613 - accuracy: 1.0000
Epoch 339/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0603 - accuracy: 1.0000
Epoch 340/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0596 - accuracy: 1.0000
Epoch 341/500
7/7 [=====

7/7 [==============================] - 0s 5ms/step - loss: 0.0263 - accuracy: 1.0000
Epoch 414/500
7/7 [==============================] - 0s 6ms/step - loss: 0.0261 - accuracy: 1.0000
Epoch 415/500
7/7 [==============================] - 0s 5ms/step - loss: 0.0258 - accuracy: 1.0000
Epoch 416/500
7/7 [==============================] - 0s 5ms/step - loss: 0.0255 - accuracy: 1.0000
Epoch 417/500
7/7 [==============================] - 0s 5ms/step - loss: 0.0253 - accuracy: 1.0000
Epoch 418/500
7/7 [==============================] - 0s 5ms/step - loss: 0.0250 - accuracy: 1.0000
Epoch 419/500
7/7 [==============================] - 0s 5ms/step - loss: 0.0248 - accuracy: 1.0000
Epoch 420/500
7/7 [==============================] - 0s 5ms/step - loss: 0.0245 - accuracy: 1.0000
Epoch 421/500
7/7 [==============================] - 0s 5ms/step - loss: 0.0243 - accuracy: 1.0000
Epoch 422/500
7/7 [==============================] - 0s 5ms/step - loss: 0.0240 - accuracy: 1.0000
Epoch 423/500
7/7 [=====

7/7 [==============================] - 0s 5ms/step - loss: 0.0126 - accuracy: 1.0000
Epoch 496/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0125 - accuracy: 1.0000
Epoch 497/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 498/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 499/500
7/7 [==============================] - 0s 5ms/step - loss: 0.0121 - accuracy: 1.0000
Epoch 500/500
7/7 [==============================] - 0s 4ms/step - loss: 0.0120 - accuracy: 1.0000


In [101]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


INFO:tensorflow:Assets written to: chat_model\assets


In [102]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User >> " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit" or inp.lower() == "q":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot >>" + Style.RESET_ALL , np.random.choice(i['responses']))
                
                
            
        
                

        



What to do if Cuts?

How do you treat a Broken Toe?

what to do if i get a Choke?

In [130]:
print(Fore.YELLOW + "Start messaging with the bot (type quit or q to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit or q to stop)!
User >> hi
1/1 [==============================] - 0s 177ms/step
ChatBot >> Hi
User >> What to do if Cuts?
1/1 [==============================] - 0s 45ms/step
ChatBot >> Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.
User >> How do you treat a Broken Toe?
1/1 [==============================] - 0s 53ms/step
ChatBot >> 1)To help decrease pain and swelling in a broken toe, elevate the foot, ice the injury, and stay off the foot. 2)Depending on the severity of the fracture, the toe may need to be put back into place (reduced), and some compound toe fractures may require surgery.3) Most broken toes heal without complications in six weeks.
User >> what to do if i get a Choke?
1/1 [=========

thanks for reading 